In [1]:
#-------------------------------------IMPORTS-------------------------------------#
import torch
import gzip
import numpy as np
from random import randint
import torch.utils.data as data_utils
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA

In [2]:
#-------------------------------------HELPER FUNCTIONS-------------------------------------#
NUM_NEGATIVE_SAMPLES=20

def N_random_values_in_list(full_list, N):
    x=0
    lower_bound  = 0
    upper_bound = len(full_list)-1
    sample_list=[]
    random_nums=[]
    while x < min(N,len(full_list)):
        random_num = randint(lower_bound, upper_bound) # inclusive range
        if random_num in random_nums:
            continue
        else:
            random_nums.append(random_num)
            x += 1
    return [full_list[i] for i in random_nums]

def convert_to_list(filename):
    if filename.endswith('gz'):
        with gzip.open(filename,'r')as f:
            text_tokens = f.readlines()
    else:
        with open(filename, 'r') as f:
            text_tokens = f.readlines()
    text_tokens = [token.replace('\n','').split('\t') for token in text_tokens]
    text_tokens = [[token[0], token[1].split(' '), token[2].split(' ')] for token in text_tokens]
                   
    return text_tokens

#Sample:question_id, similar_question_id, negative_question_id
def convert_to_samples(filename):
    my_list=convert_to_list(filename)
    new_samples=[]
    for original_sample in my_list:
        for similar in original_sample[1]:
            random_negative_samples = N_random_values_in_list(original_sample[2],NUM_NEGATIVE_SAMPLES)
            new_samples.append([original_sample[0], similar, random_negative_samples])# change this to include all negative 
                                                                                     # examples later
    return new_samples
def make_lookup_table_for_training_data(filename):
    lookup={}
    text_token_list=convert_to_list(filename)
    for token in text_token_list:
        lookup[token[0]] = {'title':token[1],'question':token[2]}
    return lookup
        
#takes  sample_ids of [[q1,p1,n1],[q2,p2,n2]....]
#outputs titles like [[q1_title, p1_title, n1_title],[q2_title,p2_title,n2_title]...]
def convert_sampleids_to_titles(sample_ids,lookup):
    #each sample_id [question_id, pos_id, [neg_ids]]
    #print type(sample_ids)==list, "first"
   
    titles = []
    for sample_id in sample_ids:
        
         #flatten list: [question_id, pos_id, [neg_ids]] --> [question_id, pos_id, neg_id1, neg_id2, ...]
        sample_id= sample_id[:2]+sample_id[2][:]
        #sample_id : question_id, similar_question_id, negative_question_id
        try:
            titles.append([lookup[str(identity)]['title'] for identity in sample_id])
           # print type(sample_id)==list
        except:
            print sample_id, "is sample id", type(sample_id)==list
    return titles
def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

def extract_features(word):
    try:
        word=remove_non_ascii(word)
        word=word.encode('utf-8')
    except:
        print(word)
    return word_to_vec.get(word,[0.0 for i in range(200)])

def find_maximum_title_and_body_length(lookup_table):
    max_len_title = -1
    max_len_question = -1
    max_len_question_id = 0
    for key, dict_val in lookup_table.iteritems():
        len_title = len(dict_val['title'])
        len_question = len(dict_val['question'])
        if len_title > max_len_title:
             max_len_title = len_title
        if len_question > max_len_question:
            max_len_question = len_question
            max_len_question_id = key
    return max_len_title, max_len_question

def title_to_feature_matrix(title_word_list):
    feature_matrix = []
    for word in title_word_list:
        word_features = extract_features(word)
        feature_matrix.append(word_features)
    #Pad the feature with zeros to ensure all inputs to the net have the same dimension
    feature_matrix += [[0.] * NUM_FEATURES_PER_WORD] * (MAX_TITLE_LENGTH - len(title_word_list))
    #print np.array(feature_matrix).T.shape
    return np.array(feature_matrix).T



#array is structured like a batch of features 50x200x38
def find_start_of_padding_for_batch(batch):
    vec_lengths_in_batch = []
    for batch_num in range(0, len(batch)):
        single_vec = batch[batch_num]
        length = find_start_of_padding_single_vec(single_vec) + 1
        vec_lengths_in_batch.append(length)
    return vec_lengths_in_batch

#batch = 200x38
def find_start_of_padding_single_vec(single_vec):
    for idx in range(len(single_vec[0])-1, -1, -1):
        if single_vec[0][idx] != 0.:
            return idx
    #if the whole sequence is 0s
    return 0
# def cosine_similarity(question_one_batch_by_features, question_two_batch_by_features):
#     cosine_similarities = []
#     for idx in range(BATCH_SIZE):
#         vec_one = question_one_batch_by_features[idx]
#         vec_two = question_two_batch_by_features[idx]
#         numerator = np.dot(vec_one, vec_two)
#         denominator = LA.norm(vec_one)*LA.norm(vec_two)
#         value = numerator/denominator
#         cosine_similarities.append(value)
#     return cosine_similarities

# def cosine_similarity_worse(question_one_batch_by_features, question_two_batch_by_features):
#     print cosine_

In [3]:
#-------------------------------------LOAD DATA-------------------------------------#

word_embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
f = gzip.open(word_embeddings, 'r')
wv_text = [ ]
lines = f.readlines()
for line in lines:
    wv_text.append(line.strip())

word_to_vec = {}

for line in wv_text:
    parts = line.split()
    word = parts[0]
    vector = np.array([float(v) for v in parts[1:]])
    word_to_vec[word] = vector
f.close()


#text_tokenized.txt.gz has id \t title \t question body
text_tokenized='askubuntu/text_tokenized.txt.gz'

#train_random.txt
#(1) the query question ID, (2) the list of similar question IDs, and (3) the list of randomly selected question IDs.
train_random_filename='askubuntu/train_random.txt'

#Each line contains (1) the query question ID, (2) the list of similar question IDs, (3) the list of 20 candidate question IDs and (4) the associated BM25 scores of these questions computed by the Lucene search engine. The second field (the set of similar questions) is a subset of the third field.
dev_filename='askubuntu/dev.txt'
test_filename='askubuntu/test.txt'

train_samples = convert_to_samples(train_random_filename)
dev_samples = convert_to_samples(dev_filename)
test_samples = convert_to_samples(test_filename)

lookup = make_lookup_table_for_training_data(text_tokenized)
train_titles_only = convert_sampleids_to_titles(train_samples, lookup)

In [5]:
import time
#-------------------------------------CREATE DATA BATCHER-------------------------------------#
#for each tuple of titles make a feature vector that is num_titles x 200 x 38
# where num_titles = 1 (target) + 1 (positive) + n (negative) 
features = []
for i in range(len(train_titles_only[:NUM_TRAINING_EXAMPLES])):   # we should include all but this is just for simplicity 
    if i%1000 == 0:
        print i

    sample = train_titles_only[i]
    
    target_title = sample[0]
    positive_title = sample[1]
    negative_titles = sample[2:]
    
#     print "target_title: {}".format(target_title)
#     print "positive title: {}".format(positive_title)
#     print "negative titles: {}".format(negative_titles)
    
    target_features = title_to_feature_matrix(target_title)
    positive_features = title_to_feature_matrix(positive_title)
    n_negative_features = [title_to_feature_matrix(negative_title) for negative_title in negative_titles]
    
#     print "Target features shape: {}".format(target_features.shape)
#     print "Positive features shape: {}".format(positive_features.shape)
#     print "Negative features[0] shape: {}".format(n_negative_features[0].shape)
#     print "Num negative features: {}".format(len(n_negative_features)) 
    
    all_features = [target_features, positive_features] + n_negative_features
    features.append(all_features)

start = time.time()
print "Done Loop"
features = np.array(features) #speed up the following operations
targets = torch.LongTensor(len(features), 1).zero_()
training_dataset = data_utils.TensorDataset(torch.FloatTensor(features), targets)
train_loader = data_utils.DataLoader(training_dataset, batch_size = BATCH_SIZE, shuffle = True)
print "Succesfully made the data batcher"

0
Done Loop
Succesfully made the data batcher


In [4]:
#-------------------------------------GLOBAL VARIABLES-------------------------------------#
NUM_TRAINING_EXAMPLES = 1000 #FOR DATA BATCHER, WHEN DEPLOYED SHOULD BE ALL TRAINING EXAMPLES

KERNEL_SIZE = 3 #MAKE SURE THIS NUMBER IS ODD SO THAT THE PADDING MAKES SENSE
PADDING = (KERNEL_SIZE - 1) / 2
INPUT_SIZE = 200
HIDDEN_SIZE = 100
LEARNING_RATE = 1e-3
NUM_EPOCHS = 25
NUM_FEATURES_PER_WORD = 200 #DO NOT CHANGE. FIXED at 200
BATCH_SIZE = 50
MAX_TITLE_LENGTH, MAX_BODY_LENGTH = find_maximum_title_and_body_length(lookup)

In [7]:
#-------------------------------------CNN-------------------------------------#
class CNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(200, hidden_dim, KERNEL_SIZE, padding = PADDING),
#             nn.ReLU()
            nn.Tanh()
        )
    def forward(self, x):
        x = self.layer1(x)
        return x

In [10]:
INPUT_DIM = (MAX_TITLE_LENGTH, NUM_FEATURES_PER_WORD)

net = CNN(INPUT_SIZE, HIDDEN_SIZE)
criterion = nn.MultiMarginLoss(p=1, margin=0.5, weight = None, size_average=True) #HAHA just put these in to look smart 
optimizer = torch.optim.Adam(net.parameters(), lr = LEARNING_RATE)

cos_torch = nn.CosineSimilarity(dim=1, eps=1e-6)

print "Beginning training..."
# ----TRAINING
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for idx,(sample, label) in enumerate(train_loader):
        #Sample shape: [50, 22, 200, 38]
        #50 - Batch size, 22 - Num questions per data point
        batch_cos_similarities = []
        batch_num = idx
        
        target_question_features = sample[:, 0]
        positive_question_features = sample[:, 1]
        N_negative_question_features = sample[:, 2:]
        
        target_question_net_output = net(Variable(target_question_features))
        positive_question_net_output = net(Variable(positive_question_features))
        N_negative_question_net_outputs = [net(Variable(negative)) for negative in N_negative_question_features]
        
        #Determine lengths to know how many vectors to take the average across.
        target_question_lengths = find_start_of_padding_for_batch(target_question_features)
        positive_question_lengths = find_start_of_padding_for_batch(positive_question_features)
        N_negative_questions_lengths = [find_start_of_padding_for_batch(negative) for negative in N_negative_question_features]
                
        batch_cos_sims = []
        #50x600x38 --> 50x600x1. Averaging across the net outputs to get a single vector for each question.
        for sample_num in range(BATCH_SIZE):
            target_question_length = target_question_lengths[sample_num]
            positive_question_length = positive_question_lengths[sample_num]
            N_negative_question_lengths = N_negative_questions_lengths[sample_num]
            N_negative_question_outputs = N_negative_question_net_outputs[sample_num]
            
            target_question_vector = target_question_net_output.data.numpy()[sample_num,:,:target_question_length].mean(axis = 1)
            positive_question_vector = positive_question_net_output.data.numpy()[sample_num,:,:positive_question_length].mean(axis = 1)
            
            N_negative_question_vectors = [negative.data.numpy()[:,:N_negative_question_lengths[idx]].mean(axis = 1) for idx, negative in enumerate(N_negative_question_outputs)]
        
            cosine_similarity_pos = np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), positive_question_vector.reshape(1, -1)))
            cosine_similarity_neg = [np.squeeze(cosine_similarity(target_question_vector.reshape(1, -1), negative.reshape(1, -1))) for negative in N_negative_question_vectors]
            
            cosine_sims = np.array([cosine_similarity_pos] + cosine_similarity_neg).flatten()
#             print cosine_sims
            batch_cos_sims.append(cosine_sims)
            if sample_num == 5:
                print "Positive question vector: {}".format(positive_question_vector)
                print "Negative question vector[0]: {}".format(N_negative_question_vectors[0])
                print "Cosine similarities for a random sample in the batch: \n{}".format(cosine_sims)
        
        optimizer.zero_grad()
        
        batch_labels = Variable(torch.from_numpy(label.numpy().squeeze()), requires_grad = True)
        loss = criterion(Variable(torch.FloatTensor(np.array(batch_cos_sims))), batch_labels)
            
        loss.backward()
            
        running_loss += loss.data[0]
            
        optimizer.step()
        print "Epoch: {}, Batch: {}, Loss: {}".format(epoch, batch_num, loss.data[0])

    print "Loss after epoch " + str(epoch) + " :" + str(running_loss)
            


Beginning training...
Positive question vector: [ 0.00305348  0.04526095 -0.0038122  -0.03348903  0.00983129 -0.02728446
  0.02944742  0.00111727  0.03187408 -0.03525553  0.00681182  0.0195019
 -0.01801823 -0.03600882 -0.01918639 -0.0121805   0.01302318  0.00557934
 -0.02558808 -0.00797534 -0.06575345 -0.01437165 -0.04329544 -0.01181246
 -0.03099145 -0.03876905 -0.03494782 -0.01977404  0.01549699 -0.00098638
  0.01345763  0.00910823 -0.03024483  0.02298164  0.00907873  0.01349907
  0.00177835 -0.03279628 -0.01706188  0.00720355  0.03502385  0.02434797
  0.04691358  0.01724326 -0.01682989 -0.02422209 -0.0203301  -0.02784275
 -0.03199393  0.02094252  0.01620647  0.00203182 -0.07192082  0.03037301
 -0.0251849   0.03703038  0.0215427   0.04811899 -0.0279613   0.00312963
  0.03211813 -0.01144157  0.00760257  0.00103984  0.02754101 -0.01368043
  0.01794758 -0.02505506  0.03092471 -0.03830152  0.02787157  0.01354861
 -0.00679153  0.02628506  0.04504211 -0.02447836 -0.0414049   0.00491713
  0.

Positive question vector: [-0.01995828  0.02170174  0.02020905 -0.02908884  0.00891085 -0.06382113
  0.05453757 -0.0321205   0.05357941 -0.0061746   0.01775406  0.02354013
 -0.03573416 -0.04814725  0.01343133 -0.01287876  0.01403258  0.0350583
 -0.00022589  0.00560176 -0.03946345 -0.03799282 -0.05170691 -0.00801655
 -0.05373139 -0.0404884  -0.01772051 -0.02777459  0.04238714  0.00069994
  0.04925744  0.01142792 -0.04912023 -0.00715157 -0.00091182  0.01629446
 -0.00664901 -0.00499871 -0.00426615  0.06239512  0.02254136  0.03338407
  0.03314077  0.04373311 -0.01887279 -0.02483429  0.00336935 -0.03600819
 -0.03342957 -0.01151879  0.02727778 -0.02417477 -0.04582496  0.01017716
 -0.01850991  0.02369701  0.02994697  0.03698331 -0.03472536  0.04171723
  0.03098645 -0.00924483  0.00214124  0.00628742  0.04450426 -0.01630668
  0.01213306  0.0133906   0.03192902 -0.01139036  0.01934363  0.01428976
 -0.00927705 -0.00035121  0.02866914 -0.03071474 -0.02544733  0.00195904
  0.00926936 -0.00815181 -

Epoch: 0, Batch: 5, Loss: 0.409551799297
Positive question vector: [ -1.10583426e-02   1.75606348e-02   3.11440583e-02  -1.91562548e-02
  -5.15384041e-03  -5.56608737e-02   3.17343995e-02   3.50292074e-03
   3.91227826e-02   3.67224542e-03   9.42722149e-03   8.54806043e-03
  -1.10768238e-02  -3.96737531e-02  -1.24682048e-02  -1.48773808e-02
  -5.62753761e-03   6.86616544e-03  -4.07532696e-03  -1.22361984e-02
  -4.11333516e-02  -3.97548825e-02  -4.29264978e-02   1.59033912e-03
  -3.65369171e-02  -5.14605045e-02  -3.06252744e-02  -4.23222631e-02
   2.99991220e-02   1.20907305e-02   3.70334014e-02   2.46662553e-02
  -3.67215127e-02   9.58951283e-03   1.16936313e-02  -2.18231063e-02
  -1.78430718e-03  -1.70692541e-02  -1.13108633e-02   2.31379122e-02
   1.71024874e-02   5.14122611e-03   5.28923050e-02   2.31470596e-02
  -2.98189037e-02  -3.14203352e-02  -8.86359252e-03  -4.53471914e-02
  -2.30759103e-02  -1.29531417e-02  -2.64068833e-03  -2.38237344e-02
  -8.27878565e-02   6.04428584e-03  

Epoch: 0, Batch: 8, Loss: 0.418787151575
Positive question vector: [  7.53812212e-03   3.94413769e-02   1.27820056e-02  -4.81321663e-03
   5.78670017e-03  -4.26594242e-02   4.63429913e-02  -6.13261946e-05
   4.12743613e-02  -1.46904476e-02  -6.63261954e-03   4.37532514e-02
  -2.25961283e-02  -1.92492940e-02  -1.28516471e-02  -1.40322438e-02
   4.09305422e-03   7.85921980e-03  -2.90863886e-02  -1.17794368e-02
  -7.72680528e-03  -1.71624627e-02  -3.90008278e-02  -3.13126133e-03
  -4.50255796e-02  -5.20277023e-02  -8.69571790e-03  -4.26893495e-02
   2.58383919e-02   2.06067506e-02   3.03535983e-02  -2.93578953e-04
  -2.72059813e-02   1.78088322e-02  -4.33553662e-03   7.22521404e-03
  -1.35214608e-02  -3.53910029e-02  -1.05961729e-02   3.19755301e-02
   3.36654335e-02  -5.76950610e-04   6.34961054e-02   1.40320128e-02
  -5.04329987e-02  -1.19661447e-02  -1.10065043e-02  -3.74665223e-02
  -1.85811929e-02   8.51549394e-03  -9.62055661e-03  -7.74733583e-03
  -8.37666690e-02   3.84163088e-03  

Positive question vector: [ -4.05499199e-03   1.22083183e-02   2.66348310e-02  -5.87105844e-03
   2.39127669e-02  -7.09876046e-02   1.28596155e-02  -1.06263077e-02
   2.52039824e-02   8.89138784e-04   2.88273878e-02   2.07279827e-02
  -1.56000350e-02  -3.16223130e-02  -1.32504720e-02  -2.08531804e-02
  -2.30864738e-04   2.68113464e-02  -3.75727145e-03  -1.53371561e-02
  -5.01332507e-02  -2.01855805e-02  -4.56235409e-02  -1.32286921e-02
  -3.47239301e-02  -5.58868423e-02  -4.28324193e-02  -3.72733176e-02
   3.89224924e-02   1.27387978e-02   3.53864431e-02   9.98240709e-03
  -6.14758357e-02   8.74305377e-04  -2.34619249e-03  -1.58046987e-02
   5.36444597e-03  -1.35588543e-02  -3.87511496e-03   4.51857224e-02
   2.56107934e-02   7.19281239e-03   4.54460718e-02   2.92797796e-02
  -1.69684347e-02  -3.18169929e-02  -2.16403743e-03  -4.72048409e-02
  -2.50364188e-02  -2.34043635e-02   3.32795270e-02  -1.34134796e-02
  -6.95252717e-02  -3.68406996e-03  -2.02680733e-02   2.24204250e-02
   1.580

Epoch: 0, Batch: 14, Loss: 0.420065611601
Positive question vector: [-0.0330102   0.02304647  0.01147723 -0.03926594 -0.0125643  -0.06135736
  0.03385504 -0.00470312  0.02971145 -0.01216932  0.01798963  0.01350706
 -0.02081605 -0.04261324 -0.00232554 -0.0153339   0.01637447  0.00699434
 -0.00925498 -0.00919778 -0.04579334 -0.03853382 -0.05106749  0.00147397
 -0.03012174 -0.04338807 -0.00142108 -0.00364099  0.02726606  0.01685162
  0.04712035  0.01677663 -0.03727866  0.01457682  0.01376092 -0.00112841
 -0.0043525  -0.00828977 -0.02310506  0.02594566  0.02079834  0.03551371
  0.03284054  0.04003544 -0.02275576 -0.03629487 -0.00318606 -0.03512165
 -0.01916271  0.01332853  0.01008727  0.00812646 -0.06268498  0.02275933
 -0.02460843  0.02784497  0.01472258  0.0322525  -0.04994664  0.03925685
  0.04589713 -0.02942978  0.00594055  0.00500989  0.04747295 -0.03014142
  0.01539244 -0.00489308  0.03028529 -0.03387542  0.01369445  0.01967575
 -0.01830179  0.00531569  0.0172069  -0.03019194 -0.0411

KeyboardInterrupt: 